In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import gspread
from apiclient import discovery
from httplib2 import Http
from oauth2client import file, client, tools

import pandas as pd
import re
import threading
import time
import os

In [ ]:
def callback(request_id, response, exception):
    if exception:
        # Handle error
        print (exception)
    else:
        print ("Permission Id: %s" % response.get('id'))

        
def get_file_list(length_delimiter,search_dir):
    os.chdir(search_dir)
    files = filter(os.path.isfile, os.listdir(search_dir))
    files = [os.path.join(search_dir, f) for f in files] # add path to each file
    files.sort(key=lambda x: os.path.getmtime(x))

    file_list = [file.split('/')[-1:] for file in files[-1:-(length_delimiter)-1:-1]]
    return([file for sublist in file_list for file in sublist])

#change the file's permissions to be shareable within company's domain
def make_shareable(id_list):
    batch = DRIVE.new_batch_http_request(callback=callback)
    domain_permission = {
        'type': 'domain',
        'role': 'reader',
        'domain': 'company domain'}
    for file_id in id_list:
        batch.add(DRIVE.permissions().create(fileId=file_id, body=domain_permission, fields='id'))
    batch.execute()

#generate shareable links from a list of files    
def create_shareable_links(file_list):
    shareable_links=[]
    for filename, mimeType in file_list:
        print(filename,mimeType)
        metadata = {'name': filename}
        if mimeType:
            metadata['mimeType'] = mimeType
        res = DRIVE.files().create(body=metadata, fields='webViewLink', media_body=filename).execute()
        shareable_links.append(res.get('webViewLink'))
        if res:
            print('Uploaded "%s"' % filename)   
    return(shareable_links)

#retrieve login information for a given vendor from master spreadsheet
def get_login(vendor):
    try:
        sheet_values = password_sheet.sheet1.get_all_values()
        for i in range(len(sheet_values)):
            if sheet_values[i][1].lower()==vendor.lower():
                username = sheet_values[i][2]
                password = sheet_values[i][3]
                return (username,password)
    except:
        raise
        

def element_exists(element_class):
    try:
        driver.find_element_by_class_name(element_class)
    except:
        return False
    return True

In [ ]:
obj = lambda: None
lmao = {"auth_host_name":'localhost', 'noauth_local_webserver':'store_true', 'auth_host_port':[8080, 8090], 'logging_level':'ERROR'}
for k, v in lmao.items():
    setattr(obj, k, v)
    
# authorization boilerplate code
SCOPES = 'https://www.googleapis.com/auth/drive'
store = file.Storage('token')
creds = store.get()

# The following will give a link if the token file does not exist to give this app permission
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_id', SCOPES)
    creds = tools.run_flow(flow, store, obj)

DRIVE = discovery.build('drive', 'v3', http=creds.authorize(Http()))

gspread_client = gspread.authorize(creds)
password_sheet = gspread_client.open('password spreadsheet')

gspread_client.list_spreadsheet_files()
sheet = gspread_client.open('Expense Report')

print('Enter month name followed by year for current expense report')
target_worksheet = input()

#switch active worksheet to target worksheet
metadata = sheet.fetch_sheet_metadata()['sheets']
for worksheet_data in metadata:
    worksheet_title = worksheet_data['properties']['title']
    if worksheet_title==target_worksheet:
        worksheet = sheet.get_worksheet(worksheet_data['properties']['index'])

In [ ]:
print('Enter start date then end date as mm/dd/yyyy for target date range')
start_date,end_date = pd.to_datetime(input()),pd.to_datetime(input())

In [ ]:
chromedriver_path = "user's chromedriver path"
downloads_path = "user's downloads path"

# Instacart

In [ ]:
email,pw = get_login('instacart')

driver = webdriver.Chrome(executable_path=chromedriver_path)
driver.set_window_size(1400,800)

driver.get('https://www.instacart.com')
driver.find_element_by_tag_name('header').find_element_by_class_name('ic-btn').click()

time.sleep(5)

email_input, password_input = driver.find_element_by_tag_name('form').find_elements_by_tag_name('input')
email_input.send_keys(email)
password_input.send_keys(pw)

driver.find_element_by_tag_name('form').find_element_by_tag_name('button').click()
time.sleep(5)
driver.get('https://instacart.com/store/account/orders')

#close pop-up message
if element_exists('toast-dismiss')==True:
    driver.find_element_by_class_name('toast-dismiss').click()

last_date = pd.to_datetime(
    "".join(driver.find_elements_by_class_name('rmq-99b9ec35')[-1]
                     .find_elements_by_tag_name('p')[1].text
                     .split(',')[:2]))

#click load more orders until all dates within range are visible (updating last date as it goes)
while start_date<last_date:
    driver.find_elements_by_tag_name('button')[-1].click()

    last_date = pd.to_datetime(
    "".join(driver.find_elements_by_class_name('rmq-99b9ec35')[-1]
                     .find_elements_by_tag_name('p')[1].text
                     .split(',')[:2]))

links=[]
dates=[]
amounts=[]
descriptions=[]

#collects all visible links within date range
for n in range(len(driver.find_elements_by_class_name('rmq-99b9ec35'))):
    order = driver.find_elements_by_class_name('rmq-99b9ec35')[n]
    order_date = pd.to_datetime("".join(order.find_elements_by_tag_name('p')[1].text.split(',')[:2]))
    if order_date>end_date:
        continue
    elif order_date<start_date:
        break
    #if order total above $0, add link to list
    elif float(order.find_elements_by_tag_name('div')[3].find_elements_by_tag_name('p')[1].text[1:])>0:
        links.append(order.find_element_by_tag_name('a').get_attribute('href'))
        amounts.append(order.find_elements_by_tag_name('div')[0].find_elements_by_tag_name('p')[5].text)
        dates.append(str(order_date).split()[0])

for link in links:
    driver.get(link)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'order-status-header'))
        ).find_element_by_class_name('receipt-link').click()
    #clicking receipt link opens new tab
    driver.switch_to.window(driver.window_handles[1])
    #click download pdf button
    driver.find_elements_by_class_name('center-page')[-1].click()
    #close receipt tab
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

FILES = [(file,False) for file in get_file_list(len(dates),downloads_path)]
share_links = create_shareable_links(FILES)
share_links = share_links[-1::-1]
order_info_list = list(zip(dates,amounts,share_links))
gspread_client.list_spreadsheet_files()
sheet = gspread_client.open('Expense Report 2018')

for info in order_info_list:
    date = info[0]
    amount=info[1]
    link = info[2]
    worksheet.append_row([date,'instacart',amount,'',link])
    
driver.quit()

# Amazon

In [ ]:
email,pw = get_login('amazon')

driver = webdriver.Chrome(executable_path=chromedriver_path)
driver.set_window_size(1400,800)

driver.get('https://www.amazon.com')

driver.find_elements_by_class_name('nav-line-1')[2].click()
driver.find_element_by_id('ap_email').send_keys(email)
driver.find_element_by_id('ap_password').send_keys(pw)
driver.find_element_by_class_name('a-button-primary').click()
driver.find_elements_by_class_name('nav-line-2')[3].click()

amazon_info = []
#iterate through each order, grabbing links for order details between range
while True:
    for element in driver.find_elements_by_class_name('order'):
        order_date = pd.to_datetime(element.find_element_by_class_name('a-size-base').text)
        if order_date<start_date:
            break
        elif order_date <= end_date:
            total = element.find_elements_by_class_name('value')[1].text[1:]
            link = (element.find_elements_by_class_name('a-link-normal')[1].get_attribute('href'))
            order_date = str(order_date).split()[0]
            amazon_info.append([order_date,total,link])
        else:
            continue
    
    #exit loop or click to next page
    if order_date<start_date:
        break
    else:
        driver.find_element_by_class_name('a-last').click()

for info in amazon_info:
    date = info[0]
    amount=info[1]
    link = info[2]
    worksheet.append_row([date,'amazon',amount,'',link])
    
driver.quit()

# Guernsey

In [ ]:
email,pw = get_login('guernsey')

driver = webdriver.Chrome(executable_path=chromedriver_path)
driver.set_window_size(1400,800)

driver.get('https://www.buyguernsey.com')

driver.find_element_by_id('form1').find_elements_by_tag_name('input')[0].send_keys(email)
driver.find_element_by_id('form1').find_elements_by_tag_name('input')[1].send_keys(pw)
driver.find_element_by_id('form1').submit()

driver.find_element_by_id('LinksRow1').find_elements_by_class_name('header-links')[1].click()
driver.find_element_by_class_name('MyAccountItem').click()

guernsey_info=[]
for row in (driver.find_element_by_id('ctl00_ContentPlaceHolder1_resultsGridView')
                    .find_elements_by_tag_name('tr')[1:]):
    try:
        order_date = row.find_elements_by_class_name('grid-data-left')[1].text
        if (pd.to_datetime(order_date)<end_date) and(pd.to_datetime(order_date)>start_date):
            row.find_elements_by_class_name('grid-data-left')[0].click()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'print-doc-link')))
            
            total = driver.find_element_by_class_name('grand-total-data').text
            link = (driver.find_element_by_class_name('print-doc-link').get_attribute('href'))
            guernsey_info.append([order_date,total,link])
            driver.back()

        else:
            continue
    except:
        break

for info in guernsey_info:
    date = info[0]
    amount=info[1]
    link = info[2]
    worksheet.append_row([date,'guernsey',amount,'',link])
    
driver.quit()

# La Colombe

In [ ]:
email,pw = get_login('la colombe')

driver = webdriver.Chrome(executable_path=chromedriver_path)
driver.set_window_size(1400,800)

driver.get('https://www.lacolombewholesale.com/')

driver.find_element_by_id('email').send_keys(email)
driver.find_element_by_id('pass').send_keys(pw)
driver.find_element_by_id('send2').click()

driver.get(driver.find_element_by_class_name('customer-menu')
                   .find_element_by_tag_name('a')
                   .get_attribute('href'))

driver.find_elements_by_class_name('item')[4].click()

colombe_info=[]
for row in driver.find_element_by_id('my-invoices-table').find_elements_by_tag_name('tr')[1:]:
    date = row.find_element_by_class_name('date').text
    if pd.to_datetime(date)>start_date and pd.to_datetime(date)<end_date:
        total = row.find_element_by_class_name('total').text
        link = row.find_element_by_class_name('action').get_attribute('href')
        colombe_info.append([date,total,link])
    else:
        continue

for info in colombe_info:
    date = info[0]
    amount=info[1]
    link = info[2]
    worksheet.append_row([date,'la colombe',amount,'coffee',link])
    
driver.quit()

# Caviar

In [ ]:
email,pw = get_login('trycaviar')

driver = webdriver.Chrome(executable_path=chromedriver_path)
driver.set_window_size(1400,800)

driver.get('https://www.trycaviar.com/')
driver.find_element_by_class_name('global-header_account').click()
driver.find_element_by_id('user_email_signin').send_keys(email)
driver.find_element_by_id('user_password_signin').send_keys(pw)
driver.find_element_by_id('js-account-flyout-signin-submit').click()
driver.find_element_by_class_name('global-header_account').click()
driver.find_elements_by_class_name('account-flyout_item-link')[1].click()

dates = []
totals = []
while True:
    for order in driver.find_elements_by_tag_name('ul')[3:-4]:

        order_elements = order.find_elements_by_tag_name('li')
        order_date = order_elements[0].text.split()[0]

        if pd.to_datetime(order_date)>=start_date:
            if pd.to_datetime(order_date)<=end_date:
                total = order_elements[2].text.split()[-1]
                order_elements[-1].find_elements_by_tag_name('a')[-1].click()
                
                driver.switch_to.window(driver.window_handles[1])
                driver.save_screenshot('caviar_'+order_date.replace('/','-')+'.png')
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                
                dates.append(order_date)
                totals.append(total)
                
            else:
                continue
        else:
            break
            
    if pd.to_datetime(order_date)<start_date:
        break
        
    driver.find_element_by_class_name('next').click()

FILES = [(file,False) for file in get_file_list(len(dates),downloads_path)]
share_links = create_shareable_links(FILES)
order_info_list = list(zip(dates,totals,share_links))

for info in order_info_list:
    date = info[0]
    amount=info[1]
    link = info[2]
    worksheet.append_row([date,'caviar',amount,'',link])
    
driver.quit()

# Ezcater

In [ ]:
email,pw = get_login('ezcater')

driver = webdriver.Chrome(executable_path=chromedriver_path)
driver.set_window_size(1400,800)

driver.get('https://www.ezcater.com')
driver.find_elements_by_class_name('global-nav__item-link')[3].click()

print("EZcater uses a captcha, complete it manually within 3 minutes to continue")
WebDriverWait(driver, 200).until(
        EC.presence_of_element_located((By.ID, 'email')))

time.sleep(5)

driver.find_element_by_id('email').send_keys(email)
driver.find_element_by_id('legacy_password').send_keys(pw)
driver.find_element_by_tag_name('button').click()
driver.find_element_by_class_name('nav-pills').find_elements_by_tag_name('li')[2].click()

dates=[]
totals=[]

for order_row in driver.find_element_by_tag_name('tbody').find_elements_by_tag_name('tr'):
    order_date = order_row.find_elements_by_tag_name('td')[3].text
    if pd.to_datetime(order_date)>=start_date:
        if pd.to_datetime(order_date)<=end_date:
            total = order_row.find_elements_by_tag_name('td')[2].text
            order_row.find_elements_by_tag_name('td')[1].click()
            dates.append(order_date)
            totals.append(total)
    elif pd.to_datetime(order_date)<start_date:
        break
    else:
        continue
        
FILES = [(file,False) for file in get_file_list(len(dates),downloads_path)]
share_links = create_shareable_links(FILES)
order_info_list = list(zip(dates,totals,share_links))

for info in order_info_list:
    date = info[0]
    amount=info[1]
    link = info[2]
    worksheet.append_row([date,'ezcater',amount,'',link])
    
driver.quit()